# Artificial Intelligence I
## Assignment 1 - Alexander Stradnic - 119377263

## Importing packages, loading dataset

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Fundamentals
import pandas as pd
import numpy as np

# Plotting
from pandas.plotting import scatter_matrix
from seaborn import scatterplot
import matplotlib.pyplot as plt

# Data splitting
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split

# Models to be used
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures

# Hyperparameter selection
from sklearn.model_selection import GridSearchCV

# Scalers, imputers, and transformers
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

# One hot encodng
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# Error calculation
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import validation_curve
from sklearn.model_selection import learning_curve


In [ ]:
df = pd.read_csv("../datasets/dataset_salaries.csv")
# Shuffle the dataset
df = df.sample(frac=1, random_state=2)
df.reset_index(drop=True, inplace=True)
df

## Filtering the dataset

First I make sure that all the salaries are all scaled correctly relative to each other (x | x * 1000 = salary).

In [ ]:

[df['basesalary'], df['totalyearlycompensation'], df['stockgrantvalue']] = np.where(df['totalyearlycompensation'] > 10000,
                                           (df['basesalary'] / 1000, df['totalyearlycompensation'] / 1000, df['stockgrantvalue'] / 1000),
                                           (df['basesalary'], df['totalyearlycompensation'], df['stockgrantvalue']))

df.describe(include="all")


In [ ]:
# scatter_matrix(df, figsize=(15, 15))

## Choosing features

I decide to select the following numeric and nominal features that I think correlate with the expected salary, including :

Numeric:
- years of experience
- years spent working at the company

Categorical:
- location <- standard location string
    - cityid <- ID number of city
    - dmaid <- Designated Market Area, which splits the US into areas by TV broadcaster area
- company
- tag (area of expertise)
- title (role of employee)
- level of employee (different labelling per company but mostly similar)

I try to avoid unfair human biases by removing gender.

In [ ]:

numeric_features = ['yearsofexperience', 'yearsatcompany']
nominal_features = ['title', 'location', 'level', 'tag', 'dmaid', 'cityid', 'company']
# nominal_features = ['location', 'tag', 'company']
features = numeric_features + nominal_features

# target_features = ['basesalary', 'bonus', 'stockgrantvalue']
target_features = ['basesalary']

cols = features + target_features

# Removing entries with no salary
df = (df[df['basesalary'] > 0]).copy()

df2 = pd.DataFrame(df, columns = cols) 

df2

In [ ]:
# Split off the test set: 20% of the dataset.
dev_df, test_df = train_test_split(df2, train_size=0.8, random_state=2)

In [ ]:
m = scatter_matrix(dev_df, figsize=(15, 15))

In [ ]:
plot = scatterplot(x="basesalary", y="yearsofexperience", data=dev_df)

In [ ]:
plot = scatterplot(x="yearsatcompany", y="yearsofexperience", data=dev_df)

In [ ]:
df2[numeric_features + target_features].corr()

In [ ]:
df2

In [ ]:
# Extract the features but leave as a DataFrame
dev_X = dev_df[features]
test_X = test_df[features]

# Target values, converted to a 1D numpy array
dev_y = dev_df["basesalary"].values
test_y = test_df["basesalary"].values

# dev_y = pd.DataFrame(dev_df,
#                          columns = target_features)
# test_y = pd.DataFrame(test_df,
#                          columns = target_features)

In [ ]:
# Defining shuffle split (75% of development data, 60% of total data)
ss = ShuffleSplit(n_splits=1, train_size=0.75, random_state=2)

I took the MetaTransformer class from the lecture slides as it allows me to choose the most accurate scaler in a given scenario.

In [ ]:
class MetaTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, transformer=None):
        self.transformer = transformer
        
    def fit(self, X, y=None):
        if self.transformer:
            self.transformer.fit(X, y)
        return self
    
    def transform(self, X, y=None):
        if self.transformer:
            return self.transformer.transform(X)
        else:
            return X

Looking at the graph for years of experience compared to the years at the company, it seems that general experience scales at a factor faster than company experience, so I created a function to try balance this relationship and pass it into the regressors.

In [ ]:
class InsertCompExpRatio(BaseEstimator, TransformerMixin):

    def __init__(self, insert=True):
        self.insert = insert
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        if self.insert:
            X["cer"] = (X["yearsatcompany"] * 3) / X["yearsofexperience"]
    
            X = X.replace( [ np.inf, -np.inf ], 0 )
        return X

I experimented with feature engineering the Log of years of experience as the yoe/basesalary seems to show a slow initial growth as yoe increases.

In [ ]:
class InsertLogExp(BaseEstimator, TransformerMixin):

    def __init__(self, insert=True):
        self.insert = insert
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        if self.insert:
            X = X.replace( [ np.inf, -np.inf, np.nan ], 0 )
            z = (X[X["yearsofexperience"]>0]).copy()
            # X["logexp"] = np.where(X["yearsofexperience"]>0, np.log(X["yearsofexperience"]), X["yearsofexperience"])

            X["logexp"] = np.log(z["yearsofexperience"])
    
            X = X.replace( [ np.inf, -np.inf, np.nan ], 0 )
        return X

## Preprocessor

In [ ]:
# Create the preprocessor
preprocessor = ColumnTransformer([
        ("num", Pipeline([("cer", InsertCompExpRatio()),
                        ("logexp", InsertLogExp()),
                        ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
                        ("scaler", MetaTransformer())]), 
                numeric_features),
        ("nom", Pipeline([("imputer", SimpleImputer(missing_values=np.nan, strategy="most_frequent")), 
                        ("binarizer", OneHotEncoder(handle_unknown="ignore"))]), 
                nominal_features)],
        remainder="passthrough")

## k-Nearest Neighbours

In [ ]:
# kNN Grid Search (using holdout)

# Create a pipeline that combines the preprocessor with kNN
knn_model = Pipeline([
    ("preprocessor", preprocessor),
    ("predictor", KNeighborsRegressor())])

# Create a dictionary of hyperparameters and values to try
param_grid = {"predictor__n_neighbors" : [i for i in range(10,21)],
                    "preprocessor__num__scaler__transformer": [StandardScaler(), MinMaxScaler(), RobustScaler()],
                    "preprocessor__num__cer__insert": [True, False],
                    "preprocessor__num__logexp__insert": [True, False]}

# Create the grid search object which will find the best hyperparameter values based on validation error
knn_gs = GridSearchCV(knn_model, param_grid, scoring="neg_mean_absolute_error", cv=ss)

# Run grid search by calling fit
knn_gs.fit(dev_X, dev_y)

# Let's see how well we did
knn_gs.best_params_, knn_gs.best_score_


In [ ]:
# kNN error estimation (n_neighbors=15, cer=False, MinMaxScaler()) on the dev set

knn_model.set_params(**knn_gs.best_params_) 

cross_val_score(knn_model, dev_X, dev_y, scoring="neg_mean_absolute_error", cv=ss)

In [ ]:
# K-NEAREST NEIGHBOURS ON THE TEST SET
knn_model.fit(dev_X, dev_y)
mean_absolute_error(test_y, knn_model.predict(test_X))

## Linear Regression

In [ ]:
# Create a pipeline that combines the preprocessor with linear regression
ols = Pipeline([
    ("preprocessor", preprocessor),
    ("predictor", LinearRegression())])

# Create a dictionary of hyperparameters
ols_param_grid = {"preprocessor__num__scaler__transformer": [StandardScaler(), MinMaxScaler(), RobustScaler()],
                    "preprocessor__num__cer__insert": [True, False],
                    "preprocessor__num__logexp__insert": [True, False]}
# Create the grid search object which will find the best hyperparameter values based on validation error
ols_gs = GridSearchCV(ols, ols_param_grid, scoring="neg_mean_absolute_error", cv=ss, refit=True)

# Run grid search by calling fit. . It will also re-train on train+validation using the best parameters.
ols_gs.fit(dev_X, dev_y)

# Let's see how well we did
ols_gs.best_params_, ols_gs.best_score_


In [ ]:
# Linear Regression on the dev set (cer=True, logexp=True, RobustScaler())
ols.set_params(**ols_gs.best_params_) 
ols.fit(dev_X, dev_y)
mean_absolute_error(dev_y, ols.predict(dev_X))

In [ ]:
# LINEAR REGRESSION ON THE TEST SET
mean_absolute_error(test_y, ols.predict(test_X))

## Ridge Regression

In [ ]:
alphas = [i for i in range(21)]
# Create a pipeline that combines the preprocessor with ridge regression
ridge = Pipeline([
    ("preprocessor", preprocessor),
    ("predictor", Ridge())])

# Create a dictionary of hyperparameters for rideg regression
ridge_param_grid = {"preprocessor__num__scaler__transformer": [StandardScaler(), MinMaxScaler(), RobustScaler()],
                    "predictor__alpha": alphas,
                    "preprocessor__num__cer__insert": [True, False],
                    "preprocessor__num__logexp__insert": [True, False]}

# Create the grid search object which will find the best hyperparameter values based on validation error
ridge_gs = GridSearchCV(ridge, ridge_param_grid, scoring="neg_mean_absolute_error", cv=10, refit=True)

# Run grid search by calling fit. It will also re-train on train+validation using the best parameters.
ridge_gs.fit(dev_X, dev_y)

# Let's see how well we did
ridge_gs.best_params_, ridge_gs.best_score_

In [ ]:
# Ridge Regression on the dev set (alpha(lambda)=9, cer=False, logexp=True, MinMaxScaler())
precomputed = True
if precomputed:
    ridge_gs.best_params_ = {'predictor__alpha': 9,
  'preprocessor__num__cer__insert': False,
  'preprocessor__num__logexp__insert': True,
  'preprocessor__num__scaler__transformer': MinMaxScaler()}
else:
    ridge.set_params(**ridge_gs.best_params_)

ridge.fit(dev_X, dev_y)
mean_absolute_error(dev_y, ridge.predict(dev_X))

In [ ]:
# RIDGE REGRESSION ON THE TEST SET
mean_absolute_error(test_y, ridge.predict(test_X))

## Stochastic Gradient Descent

In [ ]:
sgd = Pipeline([
    ("preprocessor", preprocessor),
    ("predictor", SGDRegressor(max_iter=1000, tol=1e-3, learning_rate="optimal"))])
sgd.fit(dev_X, dev_y)

# Create a dictionary of hyperparameters
sgd_param_grid = {"preprocessor__num__scaler__transformer": [StandardScaler(), MinMaxScaler(), RobustScaler()],
                    "preprocessor__num__cer__insert": [True, False],
                    "preprocessor__num__logexp__insert": [True, False],
                    "predictor__alpha": [10**i for i in range(-5, 1)]}
# Create the grid search object which will find the best hyperparameter values based on validation error
sgd_gs = GridSearchCV(sgd, sgd_param_grid, scoring="neg_mean_absolute_error", cv=ss, refit=True)

In [ ]:
# Run grid search by calling fit. . It will also re-train on train+validation using the best parameters.
sgd_gs.fit(dev_X, dev_y)

# Let's see how well we did
sgd_gs.best_params_, sgd_gs.best_score_
# Linear Regression (gradient descent) on the dev set (cer=True, logexp=True, RobustScaler())
sgd.set_params(**sgd_gs.best_params_) 
sgd.fit(dev_X, dev_y)
mean_absolute_error(dev_y, sgd.predict(dev_X))

(more tweaking is needed here)

In [ ]:
# GRADIENT DESCENT LINEAR REGRESSION ON THE TEST SET
mean_absolute_error(test_y, sgd.predict(test_X))

## Experimentation with Polynomial Regression

I tried some polynomial functions to see if I could approach or exceed the accuracy of standard linear regression, without excessive computation.

In [ ]:
exp_nominals = ["company", "tag", "title"]
exp_features = exp_nominals + numeric_features
cols = exp_features + ["basesalary"]
exp_df = pd.DataFrame(df2, columns = cols)

# Split off the test set: 20% of the dataset.
exp_dev_df, exp_test_df = train_test_split(exp_df, train_size=0.8, random_state=2)

# Extract the features but leave as a DataFrame
exp_dev_X = exp_dev_df[exp_features]
exp_test_X = exp_test_df[exp_features]

# Target values, converted to a 1D numpy array
exp_dev_y = exp_dev_df["basesalary"].values
exp_test_y = exp_test_df["basesalary"].values


preprocessor = ColumnTransformer([
        ("nom", Pipeline([("imputer", SimpleImputer(missing_values=np.nan, strategy="most_frequent")), 
                        ("binarizer", OneHotEncoder(handle_unknown="ignore", sparse=True))]), 
                exp_nominals)],
        remainder="passthrough")

quadratic_model = Pipeline([
        ("preprocessor", preprocessor),
    ("poly", PolynomialFeatures(degree=2, include_bias=False)),
    ("predictor", LinearRegression())
])

# np.mean(cross_val_score(quadratic_model, exp_dev_X, exp_dev_y, scoring="neg_mean_absolute_error", cv=10, n_jobs=-1))

-> -52.258330920038404

The code took half an hour to run on 16 threads at full speed and yet produced an output worse than the linear regressors/kNN. Clearly the data is not quadratic.

In [ ]:
quadratic_model.fit(exp_dev_X, exp_dev_y)

y_predicted = quadratic_model.predict(exp_test_X)

In [ ]:
def sct_plt():
    fig = plt.figure()
    plt.xlabel("Feature")
    plt.ylabel("y")
    # plt.ylim(-4, 14)
    plt.scatter(exp_dev_X["yearsofexperience"], exp_dev_y, color = "green")

In [ ]:
sct_plt()
plt.plot(exp_test_X["yearsofexperience"], y_predicted, color = "blue")
plt.show()

As visible from the chart, the quadratic model significantly overfits the data.

# Conclusion

Ridge regression has produced the most accurate results by a small margin followed by Linear Regression and kNN. However, it is clear that the cleanliness of the set plays a large part, and a balance between clean data and amount of data has to be found.